In [1]:
import pandas as pd
import numpy as np
import torch
DATA = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00448/carbon_nanotubes.csv',sep = ';', decimal=",")
#This is the carbon nanotubes data.
DATA = DATA.astype(float)
#print(DATA)
#print(DATA)
DATA = DATA.to_numpy()
DATA = torch.from_numpy(DATA)
#print(DATA)
np.random.seed(42069)

DATA = DATA.type(torch.float32)

data_mean = DATA.mean(axis=1)[0:8]
data_std = DATA.std(axis=1)[0:8]
DEVICE = torch.device("cuda:1")

#DATA[:,7] = 5*DATA[:,1] + 3*DATA[:,6] + 2*DATA[:,3] + 1*DATA[:,2] 
DATA[:,0:8] = (DATA[:,0:8]  - data_mean)/data_std
#print('DATA normalized:',DATA,'len(DATA) normalized:',len(DATA),'mean dtrain normalized:',DATA.mean(axis=0)[2])
tr_ids = np.random.choice(10721, 6000, replace = False)

dTRAIN_CARBON = DATA[tr_ids,:]
dTEST_CARBON = DATA[-tr_ids,:]

print(DATA)

import math
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from tensorboardX import SummaryWriter
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm, trange
import pandas as pd
import time
import mpmath
import os
import importlib

COND_OPT = False
CLASSES = 1
# TRAIN_EPOCHS = 250
SAMPLES = 1
TEST_SAMPLES = 10
TEMPER = 0.001
TEMPER_PRIOR = 0.001
pepochs = 30
epochs =10
TEST_BATCH_SIZE = 6000
BATCH_SIZE = 100

import test_ensemble
importlib.reload(test_ensemble)
import FVMF
importlib.reload(FVMF)

data_shape = (0,7,7,8)
l1shape=(7, 7)
l2shape=(7, 7)
l3shape=(7, 7)
l4shape=(7, 1)
layershapes = [l1shape, l2shape, l3shape, l4shape]

trtimes  = []


for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    net5= FVMF.BayesianNetwork(#w_mu = w_mu, b_mu = b_mu, 
                                #w_mu = None, b_mu = None,
                                #w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                dtrain=dTRAIN_CARBON, dtest=dTEST_CARBON,
                                VD='Gaussian',
                                #b_kappa=torch.Tensor(1).uniform_(3,3.1),
                                #w_kappa=torch.Tensor(1).uniform_(5,5.1),
                                Temper = 0.1,
                                classification='Regression')
    
    #for j,p in enumerate(net2.l1.parameters()):    
    #    p.requires_grad_(False)
    #    
    #for j,p in enumerate(net2.l2.parameters()):
    #    p.requires_grad_(False)
    
    optimizer = optim.Adam(net5.parameters(), lr=0.0007)
    
    
    for epoch in range(epochs):
        train = FVMF.train(net5, dTRAIN_CARBON, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100,
                                    shape = data_shape,CLASSES = 1)    
        trtimes.append(train[1].detach().cpu().numpy())
        #print('max:',net4.weight_mu[1].max())
        #print('norm:',torch.norm(net4.weight_mu[1]))

    loss, outputs, output = test_ensemble.test_ensemble(net5,dTEST_CARBON,TEST_SAMPLES,TEST_BATCH_SIZE,TEST_BATCH_SIZE,
                                      CLASSES,DEVICE,shape = data_shape,classification = False,plot = True)
    print('Gaussian 10 epoch')
    #np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")

epochs = 30
for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    net5= FVMF.BayesianNetwork(#w_mu = w_mu, b_mu = b_mu, 
                                #w_mu = None, b_mu = None,
                                #w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                dtrain=dTRAIN_CARBON, dtest=dTEST_CARBON,
                                VD='Gaussian',
                                #b_kappa=torch.Tensor(1).uniform_(3,3.1),
                                #w_kappa=torch.Tensor(1).uniform_(5,5.1),
                                Temper = 0.1,
                                classification='Regression')
    
    #for j,p in enumerate(net2.l1.parameters()):    
    #    p.requires_grad_(False)
    #    
    #for j,p in enumerate(net2.l2.parameters()):
    #    p.requires_grad_(False)
    
    optimizer = optim.Adam(net5.parameters(), lr=0.0007)
    
    
    for epoch in range(epochs):
        train = FVMF.train(net5, dTRAIN_CARBON, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100,
                                    shape = data_shape,CLASSES = 1)    
        trtimes.append(train[1].detach().cpu().numpy())
        #print('max:',net4.weight_mu[1].max())
        #print('norm:',torch.norm(net4.weight_mu[1]))
    
    loss, outputs, output = test_ensemble.test_ensemble(net5,dTEST_CARBON,TEST_SAMPLES,TEST_BATCH_SIZE,TEST_BATCH_SIZE,
                                      CLASSES,DEVICE,shape = data_shape,classification = False,plot = True)
    print('Gaussian 30 epoch')

epochs = 50
for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    net5= FVMF.BayesianNetwork(#w_mu = w_mu, b_mu = b_mu, 
                                #w_mu = None, b_mu = None,
                                #w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                dtrain=dTRAIN_CARBON, dtest=dTEST_CARBON,
                                VD='Gaussian',
                                #b_kappa=torch.Tensor(1).uniform_(3,3.1),
                                #w_kappa=torch.Tensor(1).uniform_(5,5.1),
                                Temper = 0.1,
                                classification='Regression')
    
    #for j,p in enumerate(net2.l1.parameters()):    
    #    p.requires_grad_(False)
    #    
    #for j,p in enumerate(net2.l2.parameters()):
    #    p.requires_grad_(False)
    
    optimizer = optim.Adam(net5.parameters(), lr=0.0007)
    
    
    for epoch in range(epochs):
        train = FVMF.train(net5, dTRAIN_CARBON, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100,
                                    shape = data_shape,CLASSES = 1)    
        trtimes.append(train[1].detach().cpu().numpy())
        #print('max:',net4.weight_mu[1].max())
        #print('norm:',torch.norm(net4.weight_mu[1]))
    
    loss, outputs, output = test_ensemble.test_ensemble(net5,dTEST_CARBON,TEST_SAMPLES,TEST_BATCH_SIZE,TEST_BATCH_SIZE,
                                      CLASSES,DEVICE,shape = data_shape,classification = False,plot = True)
    print('Gaussian 50 epoch')

trtimes  = []
epochs = 30

#Note, for this regression task, the last 7 to 1 layer has a Gaussian VD, where we kill the prior and simply optimize with MLE.
for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    vMFRegression30= FVMF.BayesianNetwork(#w_mu = w_mu, b_mu = b_mu, 
                                #w_mu = None, b_mu = None,
                                #w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                dtrain=dTRAIN_CARBON, dtest=dTEST_CARBON,
                                VD='vmf',
                                b_kappa=torch.Tensor(1).uniform_(3,3.1),
                                w_kappa=torch.Tensor(1).uniform_(7.5,7.6),
                                Temper = 0,classification='Regression',NODEFORCE =False)
    
    #for j,p in enumerate(net2.l1.parameters()):
    #    p.requires_grad_(False)
    #    
    #for j,p in enumerate(net2.l2.parameters()):
    #    p.requires_grad_(False)
    
    optimizer = optim.Adam(vMFRegression30.parameters(), lr=0.11)
    
    
    for epoch in range(epochs):
        train = FVMF.train(vMFRegression30, dTRAIN_CARBON, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100,
                                    shape = data_shape,CLASSES = 1)    
        trtimes.append(train[1].detach().cpu().numpy())
        print('max:',vMFRegression30.weight_mu[1].max())
        print('norm:',torch.norm(vMFRegression30.weight_mu[1]))

    loss, outputs, output = test_ensemble.test_ensemble(vMFRegression30,dTEST_CARBON,TEST_SAMPLES,TEST_BATCH_SIZE,TEST_BATCH_SIZE,
                                      CLASSES,DEVICE,shape = data_shape,classification = False,plot = True)
    print('vMF 30 epoch')

trtimes  = []
epochs = 50


#This test crashes. Likely due to the Ghost Mu's.
##Note, for this regression task, the last 7 to 1 layer has a Gaussian VD, where we kill the prior and simply optimize with MLE.
#for i in range(0, 1):
#    print(i)
#    torch.manual_seed(i)
#    vMFRegression50= FVMF.BayesianNetwork(#w_mu = w_mu, b_mu = b_mu, 
#                                #w_mu = None, b_mu = None,
#                                #w_mu = w_mu_nodewise, b_mu = b_mu,
#                                layershapes = layershapes,
#                                dtrain=dTRAIN_CARBON, dtest=dTEST_CARBON,
#                                VD='vmf',
#                                b_kappa=torch.Tensor(1).uniform_(3,3.1),
#                                w_kappa=torch.Tensor(1).uniform_(7.5,7.6),
#                                Temper = 0,classification='Regression',NODEFORCE =False)
#    
#    #for j,p in enumerate(net2.l1.parameters()):
#    #    p.requires_grad_(False)
#    #    
#    #for j,p in enumerate(net2.l2.parameters()):
#    #    p.requires_grad_(False)
#    
#    optimizer = optim.Adam(vMFRegression.parameters(), lr=0.11)
#    
#    
#    for epoch in range(epochs):
#        train = FVMF.train(vMFRegression50, dTRAIN_CARBON, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100,
#                                    shape = data_shape,CLASSES = 1)    
#        trtimes.append(train[1].detach().cpu().numpy())
#        print('max:',vMFRegression30.weight_mu[1].max())
#        print('norm:',torch.norm(vMFRegression30.weight_mu[1]))
#
#    loss, outputs, output = test_ensemble.test_ensemble(vMFRegression50,dTEST_CARBON,TEST_SAMPLES,TEST_BATCH_SIZE,TEST_BATCH_SIZE,
#                                      CLASSES,DEVICE,shape = data_shape,classification = False,plot = True)
#    print('vMF 50 epoch')
#
#trtimes  = []


#Note, for this regression task, the last 7 to 1 layer has a Gaussian VD, where we kill the prior and simply optimize with MLE.
for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    vMFRegression= FVMF.BayesianNetwork(#w_mu = w_mu, b_mu = b_mu, 
                                #w_mu = None, b_mu = None,
                                #w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                dtrain=dTRAIN_CARBON, dtest=dTEST_CARBON,
                                VD='vmf',
                                b_kappa=torch.Tensor(1).uniform_(3,3.1),
                                w_kappa=torch.Tensor(1).uniform_(7.5,7.6),
                                Temper = 0,classification='Regression',NODEFORCE =False)
    
    #for j,p in enumerate(net2.l1.parameters()):
    #    p.requires_grad_(False)
    #    
    #for j,p in enumerate(net2.l2.parameters()):
    #    p.requires_grad_(False)
    
    optimizer = optim.Adam(vMFRegression.parameters(), lr=0.11)
    
    
    for epoch in range(epochs):
        train = FVMF.train(vMFRegression, dTRAIN_CARBON, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100,
                                    shape = data_shape,CLASSES = 1)    
        trtimes.append(train[1].detach().cpu().numpy())
        print('max:',vMFRegression.weight_mu[1].max())
        print('norm:',torch.norm(vMFRegression.weight_mu[1]))

    loss, outputs, output = test_ensemble.test_ensemble(vMFRegression,dTEST_CARBON,TEST_SAMPLES,TEST_BATCH_SIZE,TEST_BATCH_SIZE,
                                      CLASSES,DEVICE,shape = data_shape,classification = False,plot = True)
    print('vMF 10 epoch')
    #np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")

GC_intensity = 1

#Kanskje er det en god idee å også arve kappaene, men halvere dem? Hva om vi også kjører på med litt normalizering av mu-ene her?
#Hva med adaptiv kappa per lag? Det er jo bare en kappa per bias-lag. GC fungerer rett og slett ikke.

w_mu_R = []
for i in range(len(vMFRegression.weight_mu)):
    #print('\n','torch.norm(net4.weight_mu[i]):',torch.norm(net4.weight_mu[i]))
    w_mu_R.append(vMFRegression.weight_mu[i]/torch.norm(vMFRegression.weight_mu[i]))
    #print('\n','norm w_mu[i]',torch.norm(w_mu[i]))
    
b_mu_R = []
for i in range(len(vMFRegression.bias_mu)):
    #print('\n','torch.norm(net4.bais_mu[i]):',torch.norm(net4.bias_mu[i]))
    b_mu_R.append(vMFRegression.bias_mu[i]/torch.norm(vMFRegression.bias_mu[i]))
    #print('\n','norm b_mu[i]',torch.norm(b_mu[i]))
    
b_rho_R= []
for i in range(len(vMFRegression.bias_rho)):
    b_rho_R.append(vMFRegression.bias_rho[i])

w_rho_R= []
for i in range(len(vMFRegression.weight_rho)):
    w_rho_R.append(vMFRegression.weight_rho[i])
#For some reason, it seems that the weights are not being properly normalized here. no actually they are, but the norm reported is the norm after 1 optimization step.

epochs = 20
for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    vMFRegression2= FVMF.BayesianNetwork(w_mu = w_mu_R, b_mu = b_mu_R, 
                                #w_mu = None, b_mu = None,
                                #w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                dtrain=dTRAIN_CARBON, dtest=dTEST_CARBON,
                                VD='vmf',
                                b_kappa= b_rho_R, #torch.Tensor(1).uniform_(1.0,3.1), 
                                w_kappa= w_rho_R, #torch.Tensor(1).uniform_(2.0,4.1), 
                                Temper = 1,classification = 'regression')
    
    #for j,p in enumerate(net2.l1.parameters()):
    #    p.requires_grad_(False)
    #    
    #for j,p in enumerate(net2.l2.parameters()):
    #    p.requires_grad_(False)
    
    optimizer = optim.Adam(vMFRegression2.parameters(), lr=0.01) #0.01 9727 acc
    
    
    for epoch in range(epochs):
        train = FVMF.train(vMFRegression2, dTRAIN_CARBON, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100,
                                    shape = data_shape,CLASSES = 1)    
        trtimes.append(train[1].detach().cpu().numpy())
        #print('max:',vMFRegression2.weight_mu[1].max())
        #print('norm:',torch.norm(vMFRegression2.weight_mu[1]))

    loss, outputs, output = test_ensemble.test_ensemble(vMFRegression2,dTEST_CARBON,TEST_SAMPLES,TEST_BATCH_SIZE,TEST_BATCH_SIZE,
                                      CLASSES,DEVICE,shape = data_shape,classification = False,plot = True)
    print('vMF 10+20 epoch NII')

#Kanskje er det en god idee å også arve kappaene, men halvere dem? Hva om vi også kjører på med litt normalizering av mu-ene her?
#Hva med adaptiv kappa per lag? Det er jo bare en kappa per bias-lag. GC fungerer rett og slett ikke.

w_mu_R = []
for i in range(len(vMFRegression2.weight_mu)):
    #print('\n','torch.norm(net4.weight_mu[i]):',torch.norm(net4.weight_mu[i]))
    w_mu_R.append(vMFRegression2.weight_mu[i]/torch.norm(vMFRegression2.weight_mu[i]))
    #print('\n','norm w_mu[i]',torch.norm(w_mu[i]))
    
b_mu_R = []
for i in range(len(vMFRegression2.bias_mu)):
    #print('\n','torch.norm(net4.bais_mu[i]):',torch.norm(net4.bias_mu[i]))
    b_mu_R.append(vMFRegression2.bias_mu[i]/torch.norm(vMFRegression2.bias_mu[i]))
    #print('\n','norm b_mu[i]',torch.norm(b_mu[i]))
    
b_rho_R= []
for i in range(len(vMFRegression2.bias_rho)):
    b_rho_R.append(vMFRegression2.bias_rho[i])

w_rho_R= []
for i in range(len(vMFRegression2.weight_rho)):
    w_rho_R.append(vMFRegression2.weight_rho[i])
#For some reason, it seems that the weights are not being properly normalized here. no actually they are, but the norm reported is the norm after 1 optimization step.

epochs = 20
for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    vMFRegression3= FVMF.BayesianNetwork(w_mu = w_mu_R, b_mu = b_mu_R, 
                                #w_mu = None, b_mu = None,
                                #w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                dtrain=dTRAIN_CARBON, dtest=dTEST_CARBON,
                                VD='vmf',
                                b_kappa= b_rho_R, #torch.Tensor(1).uniform_(1.0,3.1), 
                                w_kappa= w_rho_R, #torch.Tensor(1).uniform_(2.0,4.1), 
                                Temper = 1,classification = 'regression')
    
    #for j,p in enumerate(net2.l1.parameters()):
    #    p.requires_grad_(False)
    #    
    #for j,p in enumerate(net2.l2.parameters()):
    #    p.requires_grad_(False)
    
    optimizer = optim.Adam(vMFRegression2.parameters(), lr=0.004) #0.01 9727 acc
    
    
    for epoch in range(epochs):
        train = FVMF.train(vMFRegression3, dTRAIN_CARBON, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100,
                                    shape = data_shape,CLASSES = 1)    
        trtimes.append(train[1].detach().cpu().numpy())
        #print('max:',vMFRegression2.weight_mu[1].max())
        #print('norm:',torch.norm(vMFRegression2.weight_mu[1]))

    loss, outputs, output = test_ensemble.test_ensemble(vMFRegression3,dTEST_CARBON,TEST_SAMPLES,TEST_BATCH_SIZE,TEST_BATCH_SIZE,
                                      CLASSES,DEVICE,shape = data_shape,classification = False,plot = True)
    print('vMF 10+20+20 epoch NII')

tensor([[ 2.0375e+00,  3.9931e-01,  1.4072e-01,  ..., -3.8084e-02,
         -5.6848e-04, -1.1847e+00],
        [ 2.0375e+00,  3.9931e-01,  2.0051e-01,  ..., -7.3575e-03,
         -1.3260e-01, -8.2473e-01],
        [ 2.0375e+00,  3.9931e-01, -1.5547e-01,  ..., -4.6846e-01,
         -8.3970e-01, -1.0649e+00],
        ...,
        [ 1.8122e+01,  9.3640e+00,  5.2303e-01,  ...,  3.3039e-01,
         -2.7718e-01,  1.5732e-01],
        [ 1.8122e+01,  9.3640e+00,  5.4044e-01,  ...,  3.5240e-01,
         -2.1551e-01,  3.5602e-01],
        [ 1.8122e+01,  9.3640e+00,  5.6962e-01,  ...,  3.8670e-01,
         -4.0853e-02,  3.9589e-01]])
GPUs are used!
FVMF RELOADED
GPUs are used!
FVMF RELOADED
0
Random Init Utilized
1
loss: tensor(1.2491, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.1212, device='cuda:1', grad_fn=<MeanBackward0>)
2
loss: tensor(1.2371, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.1101, device='cuda:1', grad_fn=<MeanBackwar

10
loss: tensor(1.0768, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.0132, device='cuda:1', grad_fn=<MeanBackward0>)
11
loss: tensor(1.0343, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.0046, device='cuda:1', grad_fn=<MeanBackward0>)
12
loss: tensor(1.0200, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.0040, device='cuda:1', grad_fn=<MeanBackward0>)
13
loss: tensor(0.9967, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.0031, device='cuda:1', grad_fn=<MeanBackward0>)
14
loss: tensor(0.9978, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.0053, device='cuda:1', grad_fn=<MeanBackward0>)
15
loss: tensor(0.9950, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.0225, device='cuda:1', grad_fn=<MeanBackward0>)
16
loss: tensor(0.9920, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.0064, device='cuda:

9
loss: tensor(0.0447, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.0447, device='cuda:1', grad_fn=<MeanBackward0>)
max: tensor(3.5952, device='cuda:1', grad_fn=<MaxBackward1>)
norm: tensor(12.5565, device='cuda:1', grad_fn=<NormBackward1>)
10
loss: tensor(0.0078, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.0078, device='cuda:1', grad_fn=<MeanBackward0>)
max: tensor(3.7024, device='cuda:1', grad_fn=<MaxBackward1>)
norm: tensor(12.5590, device='cuda:1', grad_fn=<NormBackward1>)
11
loss: tensor(0.1408, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.1408, device='cuda:1', grad_fn=<MeanBackward0>)
max: tensor(3.6174, device='cuda:1', grad_fn=<MaxBackward1>)
norm: tensor(12.6328, device='cuda:1', grad_fn=<NormBackward1>)
12
loss: tensor(0.0231, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.0231, device='cuda:1', grad_fn=<MeanBackward0>)
max: tensor(3.7358, device='cuda:1', g

9
loss: tensor(0.0447, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.0447, device='cuda:1', grad_fn=<MeanBackward0>)
max: tensor(3.5952, device='cuda:1', grad_fn=<MaxBackward1>)
norm: tensor(12.5565, device='cuda:1', grad_fn=<NormBackward1>)
10
loss: tensor(0.0078, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.0078, device='cuda:1', grad_fn=<MeanBackward0>)
max: tensor(3.7024, device='cuda:1', grad_fn=<MaxBackward1>)
norm: tensor(12.5590, device='cuda:1', grad_fn=<NormBackward1>)
11
loss: tensor(0.1408, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.1408, device='cuda:1', grad_fn=<MeanBackward0>)
max: tensor(3.6174, device='cuda:1', grad_fn=<MaxBackward1>)
norm: tensor(12.6328, device='cuda:1', grad_fn=<NormBackward1>)
12
loss: tensor(0.0231, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.0231, device='cuda:1', grad_fn=<MeanBackward0>)
max: tensor(3.7358, device='cuda:1', g

40
loss: tensor(0.0112, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.0112, device='cuda:1', grad_fn=<MeanBackward0>)
max: tensor(4.8043, device='cuda:1', grad_fn=<MaxBackward1>)
norm: tensor(16.6114, device='cuda:1', grad_fn=<NormBackward1>)
41
loss: tensor(0.0265, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.0265, device='cuda:1', grad_fn=<MeanBackward0>)
max: tensor(4.7676, device='cuda:1', grad_fn=<MaxBackward1>)
norm: tensor(16.9864, device='cuda:1', grad_fn=<NormBackward1>)
42
loss: tensor(0.0567, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.0567, device='cuda:1', grad_fn=<MeanBackward0>)
max: tensor(5.2232, device='cuda:1', grad_fn=<MaxBackward1>)
norm: tensor(17.7289, device='cuda:1', grad_fn=<NormBackward1>)
43
loss: tensor(0.0322, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.0322, device='cuda:1', grad_fn=<MeanBackward0>)
max: tensor(5.2073, device='cuda:1', 

15
loss: tensor(5.7819, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.0097, device='cuda:1', grad_fn=<MeanBackward0>)
16
loss: tensor(5.6054, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.0076, device='cuda:1', grad_fn=<MeanBackward0>)
17
loss: tensor(5.9940, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.0033, device='cuda:1', grad_fn=<MeanBackward0>)
18
loss: tensor(5.3382, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.0053, device='cuda:1', grad_fn=<MeanBackward0>)
19
loss: tensor(5.6498, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.0173, device='cuda:1', grad_fn=<MeanBackward0>)
20
loss: tensor(5.7898, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(0.0112, device='cuda:1', grad_fn=<MeanBackward0>)

 This is the TSS 1400.4136962890625 This is the SS_res: 28.397449493408203 
 This is the R^2 on testset: tensor(0.979